In [12]:
from folium import plugins
import pandas as pd
import folium as fl
import numpy as np
import math as mt
import json
import geopandas as gp
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geojson
import requests

In [4]:
df = pd.read_excel('Clusters.xlsx')

In [7]:
df.columns

Index(['Unnamed: 0', 'indice', 'Órgão Registro', 'Ano', 'Hora Minuto', 'Placa',
       'Marca', 'Município', 'Data Fato', 'Nome Rua', 'Nro', 'Fato',
       'ano recup', 'Hora recup', 'Mun recup', 'Dt recup', 'fato', 'x roubo',
       'y roubo', 'x recuperacao', 'y recuperacao', 'Clusters'],
      dtype='object')

In [10]:
df1= df.loc[(pd.notna(df.Clusters))]

In [140]:
with open('Grade_Teste_1x1_SIRGAS.txt', 'r') as txt:
    grade =json.load(txt)
    
with open('Grade_POA_1x1_SIRGAS.txt', 'r') as txt1:
    gradeOtimizada =json.load(txt1)    

In [ ]:
#acessa até o id de uma feature específica
grade['features'][0]['properties']['id']
#acessa até o array de coordenadas que formama um polígono
grade['features'][0]['geometry']['coordinates'][0]

In [141]:
#mapa com grades
m = fl.Map(location=[-30.013360,-51.177189],zoom_start=10)
fl.GeoJson(gradeOtimizada).add_to(m)
m.save('testegrade.html')

In [142]:
arrayIdPol = []
arrayPoligonos = []

for feature in gradeOtimizada['features']:
    arrayIdPol.append(feature['properties']['id'])
    arrayPoligonos.append(Polygon(feature['geometry']['coordinates'][0]))

In [143]:
dfPoligons = pd.DataFrame({'id_Pol':arrayIdPol,'Poligonos':arrayPoligonos})

In [145]:
len(dfPoligons)

3742

In [146]:
#quadrantes de roubo
import time
inicio = time.time()

refIdPol=[]
cont=0
for x,y in zip(df1['x roubo'],df1['y roubo']):
    for ref,pol in zip(dfPoligons.id_Pol,dfPoligons.Poligonos):
        if pol.contains(Point(x,y)):
            refIdPol.append(ref)
            break

fim = time.time()
print(fim - inicio)

364.8627510070801


In [147]:
len(refIdPol)

31992

In [148]:
df2=df1.drop(columns=['Unnamed: 0', 'indice'])
df2['Quadrante'] = refIdPol

In [153]:
#quadrantes de recuperação
import time
inicio = time.time()

refIdPol=[]
cont=0
for x,y in zip(df1['x recuperacao'],df1['y recuperacao']):
    for ref,pol in zip(dfPoligons.id_Pol,dfPoligons.Poligonos):
        if pol.contains(Point(x,y)):
            refIdPol.append(ref)
            break
        else:
            refIdPol.append(0)

In [150]:
fim = time.time()
print(fim, inicio , fim - inicio)

1575403553.8010764 1575403163.306407 390.49466943740845


In [155]:
len(refIdPol)

31219983

In [154]:
df2['Quadrante_Recup'] = refIdPol

ValueError: Length of values does not match length of index

In [139]:
with pd.ExcelWriter('quadrantes.xlsx') as writer: 
    df2.to_excel(writer, sheet_name='Base')

In [114]:
#acessa até o id de uma feature específica
grade['features'][0]['properties']['id']
#acessa até o array de coordenadas que formama um polígono
grade['features'][0]['geometry']['coordinates'][0]

[[-51.58243164699996, -29.637005753811472],
 [-51.57268164699996, -29.637005753811472],
 [-51.57268164699996, -29.646755753811473],
 [-51.58243164699996, -29.646755753811473],
 [-51.58243164699996, -29.637005753811472]]

In [ ]:
mapaPoa = fl.Map(location=[-30.013360,-51.177189],zoom_start=10)
latlong_ro=[]
latlong_rc=[]

#preenche array com quantidade de clusters que teremos
latlongro=[]
for x in range(n1):
    latlong_ro.append([[x,x]])
    latlong_rc.append([[x,x]])
    
for xro,yro,xrc,yrc,clust in zip(dfPoa['x roubo'],dfPoa['y roubo'],dfPoa['x recuperacao'],dfPoa['y recuperacao'],dfPoa['Clusters']):
    latlong_ro[clust].append([yro, xro])
    latlong_rc[clust].append([yrc, xrc])
    
aux = 0
for z,w in zip(latlong_ro, latlong_rc):
    layRo = 'Roubo' + str(aux)
    layRc = 'Recup.' + str(aux)
    aux= aux +1
    mapaPoa.add_child(plugins.HeatMap(z[1:],name =layRo, radius = 16))                   
    mapaPoa.add_child(plugins.HeatMap(w[1:],name =layRc, radius = 16))
                      
fl.LayerControl().add_to(mapaPoa)                       
mapaPoa
mapaPoa.save('map.html')

In [ ]:
mapa = folium.Map(location=[-29.923361,-51.177189],zoom_start=10)
i=0
for xro,yro,xrc,yrc in zip(dfC5['x roubo'],dfC5['y roubo'],dfC5['x recuperacao'],dfC5['y recuperacao']):
    folium.Marker(location=[yro, xro], icon = folium.Icon(color='green')).add_to(mapa)
    folium.Marker(location=[yrc, xrc], icon = folium.Icon(color='red')).add_to(mapa)
    i = i+1
    if i ==100:
        break
#folium.Marker(location=[-30.048990,-51.215390], icon = folium.Icon(color='green')).add_to(mapa)    
    #folium.Marker(location=[xrc, yrc], icon = folium.Icon(color='green')).add_to(mapa)
mapa